# Don't fear the grepper! (Part 1)

This blog post provides an introduction to the [Raku Programming Language](https://raku.org) and its [`grep` functionality](https://docs.raku.org/routine/grep#(List)\_routine_grep).  It does not require any specific knowledge about the Raku Programming Language, although being familiar with basic `grep` functionality (of the unix utility), is recommended!

The `grep` functionality comes in two flavours in Raku: a procedural (`sub`) version, and an object oriented (`method`) version.  Since everything in Raku is an object (or can be thought of as one), and I personally mostly prefer the object oriented way, I will be discussing only the `method` way of using `grep` and friends.

## A simple example

Let's start with a simple question: Show the even numbers between 1 and 10.  That should be easy enough.

We create an array with the numbers 1 through 10.  And we put the even numbers in another array, which we can do this with the `grep` method.  The `grep` method takes a piece of code as the argument: this will then be repeatedly called with a value to determine whether or not that value should be included.

In [3]:
sub is-even($number) {    # returns True if given number is even
    return $number %% 2;  # %% is the "evenly divisible" operator
}
my @whole = 1..10;                 # fill up the array
my @even = @whole.grep(&is-even);  # fill array with even numbers
say @even;  # [2 4 6 8 10];        # show the result

[2 4 6 8 10]


Let's dissect this piece of code: the first 3 lines contain the definition of a subroutine called "is-even".  It expects a single argument `$number`, and returns either `False` or `True` depending on whether the given value is even or not.

The fourth line defines (`my`) and initializes (`=`) an array called `@whole` with the numbers `1` through `10`.

The fifth line defines (`my`) and initializes (`=`) an array called `@even` with the result of calling the method `grep` (with the subroutine `even` as the argument) on the `@whole` array.

The sixth line shows the contents of the `@even` array.

So what should we learn about Raku from this example?

- subroutines are made with `sub`
- variables with a single value have a `$` prefix (usually referred to as a `sigil`)
- values are returned with `return` from a subroutine
- arrays have a `@` prefix (`sigil`)
- calling a method is done by placing a period `.` between object and method name
- a subroutine can be referred to by prefixing its name with `&` (sigil)

## First simplification

However, that seems like a lot of code to obtain the simple answer to the question: "Show the even numbers between 1 and 10 inclusive".  So let's start by simplifying this code:

In [4]:
sub is-even($number) { $number %% 2 }  # return not needed
say (1..10).grep(&is-even);            # also works on ranges

(2 4 6 8 10)


Note that we removed the `return` from the subroutine declaration.  By default, the last expression evaluated in any block of code (recognizable by the curly braces `{` `}`) will be returned automatically.

Also note that we removed the use of arrays altogether: `(1..10)` in Raku is a [`Range` object](https://docs.raku.org/type/Range) that can also have the `grep` method called on it.

And the `say` subroutine will take any expression as its argument, so we're good here as well.

## Second simplification

However, we can even further simplify this code.  The subroutine "is-even" doesn't actually need a name, it just needs to be a piece of code.  A further simplification would thus be:

In [5]:
-> $number { $number %% 2 }

-> $number { #`(Block|3257666866632) ... }

In Raku this is referred to as a ["pointy block"](https://docs.raku.org/language/functions#index-entry-pointy_blocks).  You could think of this as subroutine without name, but there's a subtle difference with a `sub`: you **can** `return` from a subroutine, but you can **not** return from just a pointy block.  If you execute a `return` in a pointy block, it will return out of the first outer subroutine.

## Third simplification

In programming one often talks about [**DRY** as in Don't Repeat Yourself](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself).  Note that in the second simplification, we removed the repeated mentions of `@whole` and `@even`.  In this third simplification, we will remove the repeated mention of `$number`:

In [6]:
{ $^number %% 2 }

-> $number { #`(Block|3257666892584) ... }

This uses the [placeholder variable](https://docs.raku.org/language/variables#The_^_twigil) feature of Raku.  It is basically a shorthand for the code of the second simplification.

But why would we need to have a name for a variable in such a simple expression anyway?  Couldn't we do that in an even shorter way?  Well, in Raku you can, thanks to something called ["whatever currying"](https://docs.raku.org/type/Whatever#index-entry-Whatever-currying):

In [7]:
* %% 2

WhateverCode.new

What?  That's it?  Yup.  Expressive, isn't it?  So the actual code to show the even numbers between 1 and 10 inclusive, becomes:

In [ ]:
say (1..10).grep(* %% 2); # (2 4 6 8 10)

## Moving the goal

Now suppose you're only interested in the first even number?  Well, you're in luck: the Raku Programming Language comes with a [`first` method](https://docs.raku.org/routine/first#(List)_routine_first), which is essentially the same as `grep`, but which stops as soon as it has the first match.  So our code would become:

In [ ]:
say (1..10).first(* %% 2);  # 2

But what if we want to have the third even number?

Remember how we used arrays in our first version?  Well, even though `(1..10).grep(* %% 2)` is not an array, it can be considered one.  Which means you can apply indexing on it!

In [ ]:
say (1..10).grep(* %% 2)[2];  # 6

The `[2]` syntax indicates indexing, and since the first element starts at index `0`, `[2]` means the third element.

Some people, coming from other languages, might argue that this would be wasteful, checking all the values in the range to only use the third value.  And they'd be right in (most) other languages.  But in the Raku Programming Language, `grep` is a lazy method, so it will only check as many values as it needs to produce the third element.

## Conclusion

This concludes the first part of the introduction to the `grep` method, and possibly to the Raku Programming Language.  Questions and comments are always welcome.  You can also drop into the [#raku-beginner channel](https://web.libera.chat/?channel=#raku-beginner) on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it!  Thank you for reading all the way to the end.
# Don't fear the grepper! (2)

This blog post is a follow-up on [Don't fear the grepper! (1)](https://dev.to/lizmat/dont-fear-the-grepper-1-1k3e), recommended to read first if you haven't already.

## A confession

When I said:

> The grep method takes a piece of code as the argument: this will then be repeatedly called with a value to determine whether or not that value should be included.

I was in fact not telling the entire truth.  The `grep` subroutine / method will take just about anything as the argument to filter on (not just a piece of code), in a process called "smart-matching".

Smart-matching basically is a form of comparison of two objects that somehow decides whether there is a match or not.  The most visible form of that is the [`~~` infix operator](https://docs.raku.org/language/operators#index-entry-smartmatch_operator), but that is basically just syntactic sugar for an underlying mechanism.

Other programming languages have also tried an implementation of smart-match.  From that, the Raku Programming Language has learned that:

1. any smart-matching must be **asymmetrical**.
2. any smart-matching must be **configurable** for each combination of objects.

## Why a ~~ b is not the same as b ~~ a

In hindsight, it seems odd that anybody thought that having smart-match be symmetrical, would be a good idea.  Let's take the example of a duck: A duck is a bird, but not all birds are ducks!  So, in pseudo code, `Duck ~~ Bird` should be True, and `Bird ~~ Duck` should be False.  Any symmetry there would be...   weird.

## Why it must be configurable

The Raku Programming Language comes with many classes of objects built in.  But any serious development of code in Raku will see new classes of objects.  Either by yourself, or as part of a module in the [Raku ecosystem](https://raku.land).  Of course, Raku can provide some sensible defaults in smart-matching, but any developer should be able to specify the behaviour of smart-matching between any custom objects and any other (core or custom) objects.

## The .ACCEPTS method
Smart-matching in Raku is implemented by calling the [`.ACCEPTS` method](https://docs.raku.org/routine/ACCEPTS).  All core classes have an `.ACCEPTS` method implemented, either directly or through inheritance.  And you can program an `.ACCEPTS` method in your own classes: that's configurability for you!

So what happens if you write:
```
$a ~~ $b
```
That construct is basically syntactic sugar for:
```
$b.ACCEPTS($a)
```
The left-hand side of the `~~` operator becomes the argument in a call to the `.ACCEPTS` method on the object on the right-hand side of the `~~` operator.

Going back to the context of the examples of `grep` in the first blog post:
```
say (1..10).grep(* %% 2); # (2 4 6 8 10)
```

What happens under the hood, is that whatever you've given as argument to `grep`, will have its `.ACCEPTS` method called repeatedly for all values given.  And since we've given a piece of code as the argument, it will call the `.ACCEPTS` method on the `Callable` object that represents the code.

And calling the `.ACCEPTS` method on a `Callable` object will execute the code with the given argument, and return whatever the result was of executing the code.  Which is then interpreted by the `grep` logic to include the value (trueish) or not (not-trueish).

Being very verbose, you could think of the above code doing:
```
my @result;
my $grepper = * %% 2;
for 1..10 -> $number {
    if $grepper.ACCEPTS($number) {
        @result.push($number);
    }
}
say @result.List; # (2 4 6 8 10)
```
Although the actual implementation of the `grep` method takes some shortcuts, and is also lazy.  But that's stuff for another series of blog posts about iterators.

## Smart-matching against types and roles
You can use smart-matching with type objects to filter out the objects of a certain type, or which do a certain role.  Let's create an array `@s` with different types of values:
```
my @s = "a", "b", 42, "c", "d", 666, 137, π;
say @s.grep(Int);      # (42 666 137)
say @s.grep(Str);      # (a b c d)
say @s.grep(Numeric);  # (42 666 137 3.141592653589793)
```
The first `grep` smart-matches against the [`Int`](https://docs.raku.org/type/Int) type object, which is the type for integer values.

The second `grep` smart-matches against the [`Str`](https://docs.raku.org/type/Str) type object, which is the type for string values.

> For the more Unicode savvy: in Raku, these are *always* normalized to NFG (Normalization Form Grapheme).  You can think of that as a [Normalization Form C](https://unicode.org/reports/tr15/#Norm_Forms) on steroids, which tries to combine characters as observed by a human, into a *single* real (or virtual) codepoint.

The third `grep` smart-matches against the [`Numeric`](https://docs.raku.org/type/Numeric) role.  Apart from the integer values, this also includes the value for `π` (which is not an integer, but is definitely numeric).  Note that you can also spell `π` as `pi` in Raku, if you're less mathematically, and more ascii oriented.

## Smart-matching against values
You can also smart-match against values.  Let's go back to the original `grep` example, but check for a single value:
```
say (1..10).grep(2);  # (2)
```
That works, because values that are smart-matched against themselves, (generally) return `True`.  But how would you express if you want to `grep` against more than one value?  Suppose we want to filter out any values that are `2` or `7`?  Well, you can!
```
say (1..10).grep( 2 | 7 );  # (2 7)
```
What magic is this?  Well, the `2 | 7` indicates a superposition of values: something that is actually more than one value at the same time.  These are called [Junctions](https://docs.raku.org/type/Junction) in Raku.

In the above example, the `Junction` consists of the values `2` and `7`.  The [`|`](https://docs.raku.org/language/operators#index-entry-Any_junction_operator) indicates that **any** of the values will do when that `Junction` is being used in an expression and the result should collapse to something trueish or non-trueish.  The `|` is basically syntactic sugar to collect the values for the [`any` function](https://docs.raku.org/routine/any).

So the above could also have been written as:
```
say (1..10).grep( any(2,7) );  # (2 7)
```
And if you do not know the values you want to smart-match on at compile time, you can also them into an array and give that to `any`:
```
my @targets = (1..10).pick(2);     # picks two random values
say (1..10).grep( any(@targets) ); # (the two values picked)
```
## Conclusion
This concludes the second part of the introduction to the `grep` method.  This refined the functionality of `grep` by introducing the concept of "smart-matching".  And also introduced the concept of "Normalization Form Grapheme" as well as the concept of the superposition of values, aka "junctions".

Questions and comments are always welcome.  You can also drop into the [#raku-beginner](https://web.libera.chat/?channel=#raku-beginner) channel on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it! Thanks again for reading all the way to the end.
# Don't fear the grepper! (3)

This blog post contains the third instalment of the Don't fear the grepper! series ([Part 1](https://dev.to/lizmat/dont-fear-the-grepper-1-1k3e), [Part 2](https://dev.to/lizmat/dont-fear-the-grepper-2-4ki5)), recommended reading if you haven't already.

## Setting the topic

In the first instalment, we saw that a subroutine could be reduced to a [pointy block](https://docs.raku.org/language/functions#index-entry-pointy_blocks):
```
sub is-even($number) { $number %% 2 }
```
And that could be simplified to:
```
-> $number { $number %% 2 }
```
and after that, it got simplified to
```
* %% 2
```
Now, this is all very cool, but it limits to what you can do in the condition.  A more complex condition could not be specified that way.  So maybe we need to look at simplification in another direction as well.

Another way to simplify the pointy block version is using [the topic variable `$_`](https://docs.raku.org/language/variables#index-entry-topic_variable).  So instead of:
```
-> $number { $number %% 2 }
```
we could write:
```
-> $_ { $_ %% 2 }
```
The topic variable `$_` is an important tool to avoid repeating yourself.  Like in any conversation, one does not need to repeat the topic over and over again.  However, you might say, in the above example we *did* repeat `$_`!  Well, in fact, you don't have to:
```
{ $_ %% 2 }
```
is an even more simplified block (which may be familiar to some of you as a ["lambda"](https://en.wikipedia.org/wiki/Anonymous_function)).  Any block with code in it, will automatically set the topic variable `$_` *inside* of that block to the argument passed to it.

One could argue that it no longer looks like a pointy block, and you'd be right.  But under the hood, it still acts as if `-> $_` was specified, and thus preserves its "pointy" behaviour.

Note that you can call still call that block as if it were a subroutine:
```
say { $_ %% 2 }(137); # False
say { $_ %% 2 }(42);  # True
```

## More on the topic

You don't need to define the `$_` variable yourself: it is automatically defined in any scope, so you can assign to it whenever you want to.
```
$_ = 42;
say $_; # 42
```

To make this clearer in code, there is actually a [`given` control construct](https://docs.raku.org/syntax/given) that will set `$_` inside the scope it defines:
```
given 42 {
    say $_; # 42
}
```
*Every* scope has its own topic, its own `$_` variable.  You could think of it like every scope has a `my $_` in it.  Fortunately, you don't have to specify that, so you don't have to repeat yourself!

Here's an example of multiple scopes and topics:
```
$_ = 666;
given 42 {
    say $_; # 42
}
say $_; # 666
```
Note that the `$_` after the `given` block retained the value it had before the `given` block.  That's because the (implicit) outer block also had its own `$_` defined in it.  No need to worry about stepping on each others toes!

## The invisible invocant

Like many things in the Raku Programming Language, there is also a method version of [`say`](https://docs.raku.org/routine/say#(Mu)_method_say).  So you could write the above code as:
```
given 42 {
    $_.say; # 42
}
```
One of the superpowers of the topic variable is that it is also the default for the invocant in method calls.  This means that if the invocant of a method call is the topic, then you don't need to specify it.  This allows you to write the previous example as:
```
given 42 {
    .say; # 42
}
```
No invocant specified: the topic is assumed.

## Using the topic in grep

So let's expand the logic inside the block of a `grep`.  In this example, we have a list of words, and we want to filter out the ones that are completely written in uppercase:
```
say <dog BIRD Cat COW>.grep({ $_ eq .uc }); # (BIRD COW)
```
Inside the block, we're converting whatever was given as argument to uppercase ([`.uc`](https://docs.raku.org/routine/uc#(Str)_routine_uc)) and then comparing that as a string ([`eq`](https://docs.raku.org/routine/eq#(Operators)_infix_eq)) with what was given as argument (`$_`).  The result is that you get just the words that were already all uppercase to begin with.

Note the syntax for creating a list of words, using `< ... >`.  This is called ["word quoting"](https://docs.raku.org/language/quoting#Word_quoting:_%3C_%3E) in Raku, and is really handy for specifying a list of words, separated by whitespace.

## Conclusion
This concludes the third part of the series, this time introducing [the topic variable `$_`](https://docs.raku.org/language/variables#index-entry-topic_variable), how it is always defined in every scope, and how it can be used as an invisible invocant to methods.  And on the fly, also got introduced to the `< ... >` word quoting feature.

Questions and comments are always welcome.  You can also drop into the [#raku-beginner](https://web.libera.chat/?channel=#raku-beginner) channel on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it! Thanks again for reading all the way to the end.
# Don't fear the grepper! (4)

This blog post contains the fourth instalment of the Don't fear the grepper! series. [Part 1](https://dev.to/lizmat/dont-fear-the-grepper-1-1k3e), [Part 2](https://dev.to/lizmat/dont-fear-the-grepper-2-4ki5), [Part 3](https://dev.to/lizmat/dont-fear-the-grepper-3-hfp) are all recommended reading, if you haven't read them already.

## Expanding the notion of grep

The `grep` method allows one to filter a list of values: either a value gets through, or it does not.  In this way, the functionality of `grep` is rather limited.

What if you would not only like to filter out unwanted values, but also would like to *adapt* an acceptable value on the fly?  Or turn a single value into multiple values?  With the `map` method, you can!

The [`map`](https://docs.raku.org/routine/map) method provides a superset of the functionality of `grep`.  But you can also use it as `grep` with a block to do the filtering (instead of using something to smart-match against).

In many ways, understanding `map` well, will make understanding a lot of aspects of the Raku Programming Language a lot easier!  So let's focus on that a bit.

## Using map as grep

Let's go back to the original example of `grep` in this series, this time using the topic variable in a block:
```
say (1..10).grep({ $_ %% 2 }); # (2 4 6 8 10)
```
You could write this using `map` as:
```
say (1..10).map({
    if $_ %% 2 {   # is it divisible by 2?
        $_         # yes, accept
    }
    else {         # not divisible by 2
        Empty      # don't accept
    }
}); # (2 4 6 8 10)
```
A little verbose, indeed.  But please hang in there!

As we've seen before with blocks, the last value evaluated will be returned by the block.  So if the condition `$_ %% 2` is True, it will return `$_`, else it will return `Empty`.  Now, what is `Empty`, you might ask?

## Slipping away

The Raku Programming Language has a number of special values that have special meanings.  One of them is `Empty`, a [`Slip`](https://docs.raku.org/type/Slip) *without* any elements.  A `Slip` is a special type of list that automatically flattens in any outer list-like structure.

So if the condition `$_ %% 2` is **not** true, an empty Slip will be put into the result of the `map`, and thus **not** produce a value.  So effectively removing the current value of `$_` from the list.

But you are not limited to slipping an empty list!  You can use the [`slip`](https://docs.raku.org/routine/slip) subroutine to create a Slip with any values you give it.  For example:
```
say (1..10).map({
    if $_ %% 2 {                 # divisible by 2?
        slip($_ - 0.5, $_ + 0.5) # produce values around it
    }
    else {                       # not divisibly by 2
        Empty                    # don't accept
    }
}); # (1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5)
```
In this example the even numbers are replaced by two values, one 0.5 less and one 0.5 more than the topic.

## Consequences of not slipping

So, what would happen if you do **not** slip these values, but just produce the two values?
```
say (1..10).map({
    if $_ %% 2 {           # divisible by 2?
        $_ - 0.5, $_ + 0.5 # produce list with values
    }
    else {                 # not divisible by 2
        Empty              # don't accept
    }
}); # ((1.5 2.5) (3.5 4.5) (5.5 6.5) (7.5 8.5) (9.5 10.5))
```
You would get a 5-element list of 2-element lists.  Instead of a single 10-element list.  In other words, the produced lists do **not** get flattened.

> Note that it was not necessary to put parentheses around the 2-element list.  In Raku the comma (aka the [`,` operator](https://docs.raku.org/language/operators#infix_,)) creates lists, not parentheses!

If that is what you wanted, then be happy. In this blog post, it is not what we wanted, so we used a `slip` instead.

## Automatic emptying

Using an `if` / `else` structure is still pretty verbose though, and yes this can be expressed in a shorter way:
```
say (1..10).map({
    if $_ %% 2 {
        slip($_ - 0.5, $_ + 0.5)
    }
}); # (1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5)
```
What?  You just removed the `else`?  Yup!  The reason this works, is that the value of a failed `if` (or `elsif` for that matter), is `Empty`.  So you don't have to specify the `else` clause explicitly!

Still, it feels this could still be shorter.  And you'd be right!  Since there is no `else` in the code anymore, you can use the "statement modifier" version of `if` (sometimes also referred to as "postfix if"):
```
say (1..10).map({
    slip($_ - .5, $_ + .5) if $_ %% 2
}); # (1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5)
```
That reduces on curly braces, but is also known to not improve readability for *some* people.  To yours truly, it feels very natural language-like.  Like everything in life, YMMV!

## Helicopter view

Whenever you're coding something to reach a certain goal, you should always think about the way you got to a solution.  And think about whether there couldn't be an easier way to reach the same result.

This is not just about efficiency of code execution, but also about whether your code shows the intent clearly, or not.  And clear intent will make it easier for you, or anybody else, now or in the future, to [grok](https://en.wikipedia.org/wiki/Grok) the source code.

In this case, producing the values 1.5 through 10.5 with an interval of 1, can be done **much** easier, because with `map` we can also *change* values unconditionally!
```
say (1..10).map({ $_ + .5 });
# (1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5)
```
Much simpler, isn't it?  It is even so simple that we can use ["whatever-currying"](https://docs.raku.org/type/Whatever#index-entry-Whatever-currying)!
```
say (1..10).map(* + .5);
# (1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5)
```
And there you have a hopefully easy way to grok the use of the `map` method in a nutshell!

## Conclusion
This concludes the fourth part of the series, this time introducing the `map` method.  And also introducing the concept of `Empty`, and `Slip` in general.  And also showing that you can have a statement modifier version of `if` if you don't need an `else` or an `elsif`.!

Questions and comments are always welcome.  You can also drop into the [#raku-beginner](https://web.libera.chat/?channel=#raku-beginner) channel on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it! Thanks again for reading all the way to the end.
# Don't fear the grepper! (5)

This blog post contains the fifth instalment of the Don't fear the grepper! series.

[Part 1](https://dev.to/lizmat/dont-fear-the-grepper-1-1k3e), [Part 2](https://dev.to/lizmat/dont-fear-the-grepper-2-4ki5), [Part 3](https://dev.to/lizmat/dont-fear-the-grepper-3-hfp), [Part 4](https://dev.to/lizmat/dont-fear-the-grepper-4-nki) are all recommended reading, if you haven't read them already.

## The next confession

Yes, another one.  In the previous blog post I said that you can decide to not accept some values in a `.map`, by returning the `Empty` value from the block.
```
say (1..12).map({
    if $_ %% 2 {   # is it divisible by 2?
        $_         # yes, accept
    }
    else {         # not divisible by 2
        Empty      # don't accept
    }
}); # (2 4 6 8 10 12)
```

There's actually another way to decide to not accept a value, and that is by using the [`next` control flow statement](https://docs.raku.org/syntax/next).  With `next`, you're actually telling `.map` to stop executing any code inside the block immediately, and start the **next** iteration.

So how would that look in the above case?
```
say (1..12).map({
    next unless $_ %% 2;  # not divisible by 2, next!
    $_                    # accept
}); # (2 4 6 8 10 12)
```
Note that using `next` will interrupt the normal flow of the program.  When it executes, it will look up the call stack and instruct the first handler capable of handling `next` to immediately continue with the next iteration.  In this case, the `.map` method has installed such a handler.

And you see I used [`unless`](https://docs.raku.org/syntax/unless), instead of [`if`](https://docs.raku.org/syntax/if) [`not`](https://docs.raku.org/routine/not) there.  I generally use `unless` only as a statement modifier, because using it with blocks generally doesn't improve readability, and therefor maintainability of any codebase.  But of course, I could also have written `next if not $_ %% 2`!

Other than the program flow interruption feature of `next`, `next` is just a subroutine that is provided by the Raku core.  So you can have multiple references to `next` in the same block.
```
say (1..12).map({
    next unless $_ %% 2;  # not divisible by 2, next!
    next unless $_ %% 3;  # not divisible by 3, next!
    $_                    # accept
}); # (6 12)
```

By using `next` you can create quite complicated logic when doing any mapping using `.map`.

## For the last time

Sometimes you want to not accept any more values in a `.map` when a certain condition fires, for instance when a certain value is seen.  Let's take one of the above examples, and make it stop when the value **7** has been encountered:
```
my $done = False;            # create flag
say (1..12).map({
    $done = True if $_ == 7; # switch flag if appropriate
    next if $done;           # we're done, next!
    next unless $_ %% 2;     # not divisible by 2, next!
    $_                       # accept
}); # (2 4 6)
```

As you can see, this is a bit of a hassle.  Fortunately, the Raku Programming Language has a solution for that in the form of the [`last` control flow statement](https://docs.raku.org/syntax/last).  So let's rewrite this example using `last`:
```
say (1..12).map({
    last if $_ == 7;     # we're done
    next unless $_ %% 2; # not divisible by 2, next!
    $_                   # accept
}); # (2 4 6)
```

Like `next`, `last` will interrupt the normal flow of the program.  When it executes, it will look up the call stack and instruct the first handler capable of handling `last` to stop iterating.  In this case, the `.map` method has installed such a handler.

Wow, that is so much easier!

## Side effects

The previous example had one interesting side-effect: setting a flag **outside** of the block inside the `.map`.  Yes, in the Raku Programming Language you can refer to variables outside of its lexical scope, as long as they are lexically "visible".  You can use this feature for instance, to keep a count of even numbers you've seen:
```
my $seen = 0;             # initialize counter
say (1..12).map({
    next unless $_ %% 2;  # not divisible by 2, next!
    $seen++;              # increment counter
    $_                    # accept
}); # (2 4 6 8 10 12)
say "$seen even numbers"; # 6 even numbers
```

As you can see, the Raku Programming Language also has a [`++` postfix operator](https://docs.raku.org/routine/++#(Operators)_postfix_++) for incrementing integer values!

But what if you're only interested in **how many** even numbers were seen, and not interested in the actual numbers themselves?  Well, that should be easy: remove the `say`, and the final `$_` in the block (as we're not interested in the actual value when returning from the block anyway).
```
my $seen = 0;             # initialize counter
(1..12).map({
    next unless $_ %% 2;  # not divisible by 2, next!
    $seen++;              # increment counter
});
say "$seen even numbers"; # 6 even numbers
```

And in that case, we might as well make the increment conditional, and lose the `next`!
```
my $seen = 0;             # initialize counter
(1..12).map({
    $seen++ if $_ %% 2;   # divisible by 2, increment!
});
say "$seen even numbers"; # 6 even numbers
```

This has now become a case in which the `.map` method **only** executes the given block for its side-effects.

## For all we know

Actually, the Raku Programming Language has a better syntax for that: the [`for` control statement](https://docs.raku.org/syntax/for):
```
my $seen = 0;             # initialize counter
for 1..12 {
    $seen++ if $_ %% 2;   # divisible by 2, increment!
}
say "$seen even numbers"; # 6 even numbers
```

Yes.  The `for` loop in Raku, is basically a `.map` of which the body is only executed for its side-effects.  They both use the same underlying iterator mechanism.  Which means that you can use `next` and `last` also in `for` loops, because it is basically a `.map` (or vice-versa, depending on how you look at it).

The underlying iterator mechanism is material for a whole separate set of blog posts, so I won't go further into that here and now.  Suffice to say that Raku attempts to unify many different concepts that appear to be different on the surface, to deeper unifying logic and syntax.

## Signature features

Remember that in the first post of this series, we saw that you could create a block taking a value and put it into a specific variable:
```
-> $number { $number %% 2 }
```

Would you be able to use that same syntax with `for`?  Yes, you can:
```
my $seen = 0;                # initialize counter
for 1..12 -> $number {
    $seen++ if $number %% 2; # divisible by 2, increment!
}
say "$seen even numbers";    # 6 even numbers
```

In fact, the `-> $number` syntax is a property of the *block*, not of the `.map` or the `for` loop!  In fact, that feature is called the [signature](https://docs.raku.org/routine/signature) property of the block.  Does this also imply that you can use that syntax in for instance an `if` statement?  Yes, you can:
```
if complicated-calcution($input) -> $result {
    say "Result of calculation: $result";
}
```

Because the `if` statement also just accepts a block, just as `.map` or `for` expect a block!

## But what about grep?

This series of blog posts has "grep" in its title.  So how does this apply to `.grep`?  Could you use `.grep` for its side-effects?  Yes, you could:
```
my $seen = 0;             # initialize counter
(1..12).grep({
    $seen++ if $_ %% 2;   # divisible by 2, increment!
});
say "$seen even numbers"; # 6 even numbers
```
But you probably shouldn't.  Because `.grep` is intended to filter out values from a list, and you're not using it for that in this example.  And using `.grep` for its side-effects only will confuse whoever will be maintaining your code in the future!  And that could be you!

## Conclusion
This concludes the fifth part of the series, this time introducing the `next` and `last` loop control flow statements.  And hopefully instilled the notion that a `for` loop is nothing but a `.map` that is only executed for its side-effects.  Also that blocks have signatures, that can be specified in many other situations in Raku code, such as with an `if`.

Questions and comments are always welcome.  You can also drop into the [#raku-beginner](https://web.libera.chat/?channel=#raku-beginner) channel on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it! Thanks again for reading all the way to the end.
# Don't fear the grepper! (6)

This is the 6th and final part of the ["Don't fear the grepper!"](https://dev.to/lizmat/dont-fear-the-grepper-1-1k3e) series.

## Storing results

In all of the previous instalments of this series of blog posts, the result of a `.grep` or `.map` operation was always immediately shown with `say`.

The `say` subroutine is supplied by the Raku core: it calls the `.gist` method on the given object(s), which is expected to give you a... *gist* (as in "the general meaning of a text").

For example:
```
say (1..5).map(* + 2);
```
shows:
```
(3 4 5 6 7)
```
Note that the gist of the result of the `.map` is shown with parentheses to give you an idea of the listiness of the result.

> All objects in Raku have a `.gist` method, inherited from `Any`.  If you don't like the gist it produces for your classes, you will need to provide [your own `method gist`](https://docs.raku.org/syntax/Overriding%20default%20gist%20method).

You can also store the results of a `.grep` or `.map` in an array:
```
my @result = (1..5).map(* + 2);
say @result;
```
which shows:
```
[3 4 5 6 7]
```
Note that this shows the result using square brackets.  That's because the `.gist` method on `Array` objects uses square brackets.  Which is otherwise all pretty straightforward.

You can even see the values as they're being calculated if you want to:
```
my @result = (1..5).map({ say "calculating $_"; $_ + 2});
say "stored";
say @result;
```
which shows:
```
calculating 1
calculating 2
calculating 3
calculating 4
calculating 5
stored
[3 4 5 6 7]
```

And if you're not interested in the complete result, you could just ask for it to show the first element.  And as indexing in Raku is zero-based, that would be index *0*:
```
my @result = (1..5).map({ say "calculating $_"; $_ + 2});
say "stored";
say @result[0];
```
which would show:
```
calculating 1
calculating 2
calculating 3
calculating 4
calculating 5
stored
3
```
In this case, all possible values were calculated and actually stored in the array `@result` even though you were only interested in the first value (the first element in the array).  Which may be costly if there are a lot of values to calculate.

## Being efficient

But couldn't you store the result in a scalar variable, and get the same result?  Yes, you can, but the flow of execution and the result would be subtly different:
```
my $result = (1..5).map({ say "calculating $_"; $_ + 2});
say "stored";
say $result;
```
which would show:
```
stored
calculating 1
calculating 2
calculating 3
calculating 4
calculating 5
(3 4 5 6 7)
```
Note that we're back to showing the final result using parentheses.  That's really because we're in fact just "gisting" (as one would say as an experienced Rakoon) the result of the `.map` like the original example.

What is more important to note, is that "stored" appears **before** you can see the values being calculated.  It almost looks like the `.map` is not getting executed, until we actually need to make a *gist* of it in order to be able to `say` it.  And you'd be right!

This is one of the properties of the Raku Programming Language: it tries to do as little as possible, and only do the stuff that's needed when its needed.

But you may ask, why did it fill the array completely in the example with `@result`?  In short, That's because it was decided that when an array is assigned to, it will keep filling until the right hand side of the assignment has been exhausted.

> Actually, it's a little more general than that, but this should be enough explanation for now

So you can think of:
```
my @result = (1..5).map({ say "calculating $_"; $_ + 2});
```
as:
```
my @result;
for (1..5).map({ say "calculating $_"; $_ + 2}) {
    @result.push($_);
}
```
It was decided that any other sort of (default) behaviour would have been too confusing for people used to other programming languages.

## It's an object

Remember:

> Everything in Raku is an object, or can appear to be one

The result of a `.map` is **also** an object.  And you can you use the [`.WHAT` method](https://docs.raku.org/language/mop#index-entry-syntax_WHAT-WHAT) to interrogate what kind of object something is.  Some examples:
```
say 42.WHAT;     # (Int)
say "foo".WHAT;  # (Str)
say (1..5).WHAT; # (Range)
```
Note that `.WHAT` returns the [type object](https://docs.raku.org/language/classtut#index-entry-type_object) (aka the class) of an object.  And the `.gist` method for type objects puts parentheses around the name as an indicator it is a type object.

So what type of object is returned by `.map`?
```
say (1..5).map({ .say; $_ + 2}).WHAT; # (Seq)
```
A [`Seq`](https://docs.raku.org/type/Seq) object. Aha!

Note that calling `.WHAT` on the `Seq` object was completely silent otherwise.  That's because it didn't execute anything.  Because it didn't need to.  Because it just interrogated meta-information about the `Seq` object.

## Indexing a Seq

So what will happen if you want to see the first element only of a `Seq` object stored in a scalar variable?  You can use the same indexing as we did on the `@result` array, because `Seq` objects understand that:
```
my $seq = (1..5).map({ say "calculating $_"; $_ + 2});
say "stored";
say $seq[0];
```
which shows:
```
stored
calculating 1
3
```
Wow.  It only calculated a single value!  Yes, here Raku could be, and actually was, as efficient as possible, because you only needed the first element.  But what if you also want the third element?
```
my $seq = (1..5).map({ say "calculating $_"; $_ + 2});
say $seq[0];
say $seq[2];
```
shows:
```
calculating 1
3
calculating 2
calculating 3
5
```
As you can see, it doesn't re-calculate the first element again.  So yes, it looks like it is cached somewhere.  And you'd be right again.  As soon as you use indexing on a `Seq`, it will create a hidden array that will be used to cache previously calculated values.  Technically, that's because the `Seq` class performs the [`PositionalBindFailover`](https://docs.raku.org/type/PositionalBindFailover) role.

## From here to infinity

It's this efficiency in Raku that allows you to actually specify [`Inf`](https://docs.raku.org/type/Num#index-entry-Inf_(definition)) or [`Whatever`](https://docs.raku.org/type/Whatever) as the end-point of the range in our example:
```
my $seq = (1..*).map({ say "calculating $_"; $_ + 2});
say $seq[0];
```
which shows:
```
calculating 1
3
```
without being busy calculating all values until the end of time or memory.

## Grep the mapper

One nice feature of `Seq`, is that you can call `.grep` (or `.map` for that matter, or vice-versa) on it as well.  So let's go all the way back to the initial example of filtering on even numbers.  In the following example, we first create a `Seq` object with `.map`, then create a new `Seq` object using `.grep` on that.  And then show the first three elements (`[0..2]`):
```
my $seq = (1..*).map({ say "map $_"; $_ + 2});
$seq = $seq.grep({
    say $_ %% 2 ?? "accept $_" !! "deny $_";
    $_ %% 2
});
say $seq[0..2];
```
which shows:
```
map 1
deny 3
map 2
accept 4
map 3
deny 5
map 4
accept 6
map 5
deny 7
map 6
accept 8
(4 6 8)
```
This shows that all values are produced one-by-one through the chain as efficiently as possible.

Now how that all works under the hood, is going to be the subject of a slightly more advanced series of blog posts, tentatively titled "A gaze of iterators".

## Conclusion

This concludes the sixth and final part of the series.

It shows that the Raku Programming Language has a `Seq` object that is responsible for producing values.  And that every object has a [`.WHAT` method](https://docs.raku.org/syntax/WHAT) that gives you the type object of an instance, and a `.gist` method.  While sneakily introducing the [ternary `??` `!!` operator](https://docs.raku.org/routine/%3F%3F%20!!).

Questions and comments are always welcome.  You can also drop into the [#raku-beginner](https://web.libera.chat/?channel=#raku-beginner) channel on Libera.chat, or on Discord if you'd like to have more immediate feedback.

I hope you liked it! Thanks again for reading all the way to the end.